In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from PIL import Image
import pandas as pd
!pip install timm
import timm  
import numpy as np

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [2]:
def path_data_axial(category,patient):
    path = "./data/images/"+category+"/axial/"+patient+".png"
    return path
def path_data_sagital(category,patient):
    path = "./data/images/"+category+"/sagital/"+patient+".png"
    return path
def path_data_coronal(category,patient):
    path = "./data/images/"+category+"/coronal/"+patient+".png"
    return path

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Load data
total_data = pd.read_csv("./total_data.csv")
total_data = total_data[(total_data["Research Group"] != "MCI")]
total_data = total_data.reset_index()
PID = total_data["Subject ID"]
Labels = total_data["Research Group"]

In [5]:
# Prepare data
data = []
new_labels = []
for label in Labels:
    if label == "CN":
        new_labels.append(0)
    elif label == "AD":
        new_labels.append(1)

In [6]:
for i in range(len(PID)):
    # Example: using axial images
    img_path = path_data_coronal(Labels[i], PID[i])  # Replace "Category_Name" with actual category
    data.append((img_path, new_labels[i]))
    
data

[('./data/images/CN/coronal/002_S_0295.png', 0),
 ('./data/images/CN/coronal/002_S_0413.png', 0),
 ('./data/images/CN/coronal/002_S_0559.png', 0),
 ('./data/images/AD/coronal/002_S_0619.png', 1),
 ('./data/images/CN/coronal/002_S_0685.png', 0),
 ('./data/images/AD/coronal/002_S_0816.png', 1),
 ('./data/images/AD/coronal/002_S_0938.png', 1),
 ('./data/images/AD/coronal/002_S_0955.png', 1),
 ('./data/images/AD/coronal/002_S_1018.png', 1),
 ('./data/images/CN/coronal/002_S_1261.png', 0),
 ('./data/images/CN/coronal/002_S_1280.png', 0),
 ('./data/images/CN/coronal/002_S_4213.png', 0),
 ('./data/images/CN/coronal/002_S_4225.png', 0),
 ('./data/images/CN/coronal/002_S_4262.png', 0),
 ('./data/images/CN/coronal/002_S_4264.png', 0),
 ('./data/images/CN/coronal/002_S_4270.png', 0),
 ('./data/images/AD/coronal/002_S_5018.png', 1),
 ('./data/images/CN/coronal/002_S_6053.png', 0),
 ('./data/images/CN/coronal/003_S_0907.png', 0),
 ('./data/images/CN/coronal/003_S_0981.png', 0),
 ('./data/images/CN/

In [7]:
# Define the data loader and transformations
class ImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path, label = self.data[index]
        img = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [8]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
# 5-Fold Cross-Validation
from sklearn.metrics import accuracy_score
import torchvision

kf = KFold(n_splits=5)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(kf.split(data)):
    # Split data
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Create data loaders
    dataset = ImageDataset(data, transform=transform)
    train_loader = DataLoader(dataset, batch_size=16, sampler=train_subsampler)
    test_loader = DataLoader(dataset, batch_size=16, sampler=test_subsampler)

    # Load a pretrained ViT model

    # 1. Get pretrained weights for ViT-Base
    pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT 

    # 2. Setup a ViT model instance with pretrained weights
    pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

    # 3. Freeze the base parameters
    for parameter in pretrained_vit.parameters():
        parameter.requires_grad = False

    # 4. Change the classifier head 
    class_names = [0,1]


    pretrained_vit.heads = nn.Linear(in_features=768, out_features=2).to(device)
    pretrained_vit_transforms = pretrained_vit_weights.transforms()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    pretrained_vit.to(device)

    # Loss Function, Optimizer

    optimizer = torch.optim.Adam(params=pretrained_vit.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()

    train_f1_scores, train_accuracies = [], []
    test_f1_scores, test_accuracies = [], []

    # Training Loop for this fold
    for epoch in range(30):  # Number of epochs
        pretrained_vit.train()
        train_targets, train_predictions = [], []
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = pretrained_vit(images)
            _, predicted = torch.max(outputs.data, 1)
            train_targets.extend(labels.cpu().numpy())
            train_predictions.extend(predicted.cpu().numpy())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        
        # Calculate training metrics
        train_f1 = f1_score(train_targets, train_predictions, average='weighted')
        train_acc = accuracy_score(train_targets, train_predictions)
        train_f1_scores.append(train_f1)
        train_accuracies.append(train_acc)

    # Evaluation for this fold
    pretrained_vit.eval()
    test_targets, test_predictions = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = pretrained_vit(images)
            _, predicted = torch.max(outputs.data, 1)
            test_targets.extend(labels.cpu().numpy())
            test_predictions.extend(predicted.cpu().numpy())

    # Calculate testing metrics
    test_f1 = f1_score(test_targets, test_predictions, average='weighted')
    test_acc = accuracy_score(test_targets, test_predictions)
    test_f1_scores.append(test_f1)
    test_accuracies.append(test_acc)

    # Print fold results
    print(f"Fold {fold + 1}")
    print(f"Training F1 Score: {np.mean(train_f1_scores)}, Training Accuracy: {np.mean(train_accuracies)}")
    print(f"Testing F1 Score: {test_f1}, Testing Accuracy: {test_acc}")

    # Store results for averaging later
    fold_results.append({
        'fold': fold + 1,
        'train_f1': np.mean(train_f1_scores),
        'train_accuracy': np.mean(train_accuracies),
        'test_f1': test_f1,
        'test_accuracy': test_acc
    })

# Average results over all folds
avg_train_f1 = np.mean([result['train_f1'] for result in fold_results])
avg_train_accuracy = np.mean([result['train_accuracy'] for result in fold_results])
avg_test_f1 = np.mean([result['test_f1'] for result in fold_results])
avg_test_accuracy = np.mean([result['test_accuracy'] for result in fold_results])

print(f"Average Training F1 Score: {avg_train_f1}, Average Training Accuracy: {avg_train_accuracy}")
print(f"Average Testing F1 Score: {avg_test_f1}, Average Testing Accuracy: {avg_test_accuracy}")

Fold 1
Training F1 Score: 0.7905194783839895, Training Accuracy: 0.790819209039548
Testing F1 Score: 0.6662813200803374, Testing Accuracy: 0.6694915254237288
Fold 2
Training F1 Score: 0.7774603728976527, Training Accuracy: 0.7790254237288133
Testing F1 Score: 0.6933840498810778, Testing Accuracy: 0.6694915254237288
Fold 3
Training F1 Score: 0.7883825011019767, Training Accuracy: 0.7932203389830509
Testing F1 Score: 0.6436781609195402, Testing Accuracy: 0.4745762711864407
Fold 4
Training F1 Score: 0.7669970401446842, Training Accuracy: 0.7685734463276835
Testing F1 Score: 0.7583524033795859, Testing Accuracy: 0.7542372881355932
Fold 5
Training F1 Score: 0.7731772613116461, Training Accuracy: 0.780508474576271
Testing F1 Score: 0.8020304568527918, Testing Accuracy: 0.6694915254237288
Average Training F1 Score: 0.7793073307679899, Average Training Accuracy: 0.7824293785310734
Average Testing F1 Score: 0.7127452782226666, Average Testing Accuracy: 0.6474576271186441
